<a href="https://colab.research.google.com/github/deep-diver/keras-sd-serving/blob/main/tf_serving_sd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Deviates from the original installation instructions.
# https://issuemode.com/issues/tensorflow/serving/92945160
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'
!dpkg -i tensorflow-model-server-universal_2.8.0_all.deb

In [ ]:
!apt install apt-transport-https curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!apt update && apt install bazel

In [3]:
!git clone https://github.com/keras-team/keras-cv.git

Cloning into 'keras-cv'...
remote: Enumerating objects: 6010, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 6010 (delta 124), reused 134 (delta 57), pack-reused 5771
Receiving objects: 100% (6010/6010), 8.95 MiB | 18.60 MiB/s, done.
Resolving deltas: 100% (3811/3811), done.


In [4]:
%cd keras-cv

/content/keras-cv


In [5]:
!git checkout tags/v0.3.5 -b dev

!python3 build_deps/configure.py

!bazel build build_pip_pkg
!bazel-bin/build_pip_pkg wheels

Switched to a new branch 'dev'

Configuring KerasCV to be built from source...
> Building only CPU ops

Build configurations successfully written to .bazelrc :

build --action_env TF_HEADER_DIR="/usr/local/lib/python3.8/dist-packages/tensorflow/include"
build --action_env TF_SHARED_LIBRARY_DIR="/usr/local/lib/python3.8/dist-packages/tensorflow"
build --action_env TF_SHARED_LIBRARY_NAME="libtensorflow_framework.so.2"
build --action_env TF_CXX11_ABI_FLAG="1"
build --action_env TF_CPLUSPLUS_VER="c++14"
build --spawn_strategy=standalone
build --strategy=Genrule=standalone
build  --experimental_repo_remote_exec
build -c opt
build --cxxopt="-D_GLIBCXX_USE_CXX11_ABI="1
build --copt=-mavx
build --cxxopt=-std=c++14
build --host_cxxopt=-std=c++14

Extracting Bazel installation...
Starting local Bazel server and connecting to it...
Loading: 0 packages loaded
Analyzing: target //:build_pip_pkg (4 packages loaded, 5 targets configured)
Analyzing: target //:build_pip_pkg (6 packages loaded, 11 targe

In [6]:
!pip install /content/keras-cv/wheels/keras_cv-*.whl

ERROR: keras_cv-*.whl is not a valid wheel filename.


In [7]:
import time
import base64
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
keras_cv.__version__

'0.3.4'

In [6]:
from keras_cv.models.stable_diffusion.text_encoder import TextEncoder
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
from keras_cv.models.stable_diffusion.decoder import Decoder

ModuleNotFoundError: ignored

In [9]:
MAX_PROMPT_LENGTH = 77
IMG_HEIGHT = 512
IMG_WIDTH = 512
 
text_encoder = TextEncoder(MAX_PROMPT_LENGTH)
diffusion_model = DiffusionModel(IMG_HEIGHT, IMG_WIDTH, MAX_PROMPT_LENGTH)
decoder = Decoder(IMG_HEIGHT, IMG_WIDTH)

198180272/198180272 [==============================] - 2s 0us/step


In [10]:
text_encoder_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_encoder.h5",
    file_hash="4789e63e07c0e54d6a34a29b45ce81ece27060c499a709d556c7755b42bb0dc4",
)
diffusion_model_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_diffusion_model.h5",
    file_hash="8799ff9763de13d7f30a683d653018e114ed24a6a819667da4f5ee10f9e805fe",
)
decoder_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_decoder.h5",
    file_hash="ad350a65cc8bc4a80c8103367e039a3329b4231c2469a1093869a345f55b1962",
)

In [11]:
text_encoder.load_weights(text_encoder_weights_fpath)
diffusion_model.load_weights(diffusion_model_weights_fpath)
decoder.load_weights(decoder_weights_fpath)

In [12]:
import tensorflow as tf
from tensorflow import keras

from keras_cv.models.stable_diffusion.clip_tokenizer import SimpleTokenizer
tokenizer = SimpleTokenizer()

def _get_pos_ids():
    return tf.convert_to_tensor([list(range(MAX_PROMPT_LENGTH))], dtype=tf.int32)

def encode_text(prompt):
    """Encodes a prompt into a latent text encoding.
    The encoding produced by this method should be used as the
    `encoded_text` parameter of `StableDiffusion.generate_image`. Encoding
    text separately from generating an image can be used to arbitrarily
    modify the text encoding priot to image generation, e.g. for walking
    between two prompts.
    Args:
        prompt: a string to encode, must be 77 tokens or shorter.
    Example:
    ```python
    from keras_cv.models import StableDiffusion
    model = StableDiffusion(img_height=512, img_width=512, jit_compile=True)
    encoded_text  = model.encode_text("Tacos at dawn")
    img = model.generate_image(encoded_text)
    ```
    """
    # Tokenize prompt (i.e. starting context)
    inputs = tokenizer.encode(prompt)
    if len(inputs) > MAX_PROMPT_LENGTH:
        raise ValueError(
            f"Prompt is too long (should be <= {MAX_PROMPT_LENGTH} tokens)"
        )
    phrase = inputs + [49407] * (MAX_PROMPT_LENGTH - len(inputs))
    phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)

    context = text_encoder.predict_on_batch([phrase, _get_pos_ids()])

    return context

encoded_text = encode_text("photograph of an astronaut riding a horse")

1356917/1356917 [==============================] - 0s 0us/step


In [13]:
inputs = tokenizer.encode("photograph of an astronaut riding a horse")
inputs

[49406, 8853, 539, 550, 18376, 6765, 320, 4558, 49407]

In [14]:
phrase = inputs + [49407] * (MAX_PROMPT_LENGTH - len(inputs))
phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)
phrase

<tf.Tensor: shape=(1, 77), dtype=int32, numpy=
array([[49406,  8853,   539,   550, 18376,  6765,   320,  4558, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
        49407, 49407, 49407, 49407, 49407]], dtype=int32)>

In [15]:
encoded_text

array([[[-0.3883766 ,  0.02294356, -0.05219662, ..., -0.48988292,
         -0.30660194,  0.06745391],
        [ 0.1156525 ,  0.4144913 ,  2.3049333 , ..., -1.7711414 ,
         -1.0141486 , -0.9208659 ],
        [-0.4335628 , -0.09044406,  0.722898  , ...,  0.8755375 ,
         -1.0948254 , -0.7246346 ],
        ...,
        [-3.080411  , -0.19198483, -0.33464575, ...,  0.5393776 ,
         -0.15574259,  0.60614014],
        [-3.0885706 , -0.24322483, -0.3512005 , ...,  0.5685484 ,
         -0.15004434,  0.5980401 ],
        [-3.0169518 , -0.24591564, -0.30276614, ...,  0.5613408 ,
         -0.10421933,  0.5855357 ]]], dtype=float32)

# Text Encoder in SavedModel

Assumes that tokens are prepared to be passed to the model with `SimpleTokenizer`

In [16]:
text_encoder.inputs

[<KerasTensor: shape=(None, 77) dtype=int32 (created by layer 'tokens')>,
 <KerasTensor: shape=(None, 77) dtype=int32 (created by layer 'positions')>]

### 🛑 can't make concrete function, current codes simply calls the model itself without concrete function. need to investigate if this is an error

In [ ]:
# 
m_call = tf.function(text_encoder.call).get_concrete_function(
        tf.TensorSpec(
            shape=[1, 77], dtype=tf.int32, name='token'
        ),
        tf.TensorSpec(
            shape=[1, 77], dtype=tf.int32, name='positions'
        )
    )

AssertionError: ignored

In [4]:
%cd /content

/content


In [27]:
from keras_cv.models.stable_diffusion.constants import _UNCONDITIONAL_TOKENS

signature_dict = { "tokens": tf.TensorSpec(shape=[], dtype=tf.int32, name="tokens"),
                   "batch_size": tf.TensorSpec(shape=[], dtype=tf.int32, name="batch_size")}

def model_exporter(model: tf.keras.Model):
    MAX_PROMPT_LENGTH = 77
    pos_ids = tf.convert_to_tensor([list(range(MAX_PROMPT_LENGTH))], dtype=tf.int32)

    @tf.function(input_signature=[signature_dict])
    def serving_fn(string_input):
        batch_size = string_input["batch_size"]

        # context
        encoded_text = model([string_input['tokens'], pos_ids])
        encoded_text = tf.squeeze(encoded_text)

        if encoded_text.shape.rank == 2:
            encoded_text = tf.repeat(
                tf.expand_dims(encoded_text, axis=0), batch_size, axis=0
            )
        context = encoded_text

        # unconditional context
        unconditional_tokens = tf.convert_to_tensor([_UNCONDITIONAL_TOKENS], dtype=tf.int32)
        unconditional_context = model([unconditional_tokens, pos_ids])        

        unconditional_context = tf.repeat(
            unconditional_context, batch_size, axis=0
        )
        return {"context": encoded_text, "unconditional_context": unconditional_context}

    return serving_fn

In [28]:
tf.saved_model.save(
    text_encoder,
    "./text_encoder/1/",
    signatures={"serving_default": model_exporter(text_encoder)},
)

In [29]:
!saved_model_cli show --dir text_encoder/1/ --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['batch_size'] tensor_info:
      dtype: DT_INT32
      shape: ()
      name: serving_default_batch_size:0
  inputs['tokens'] tensor_info:
      dtype: DT_INT32
      shape: ()
      name: serving_default_tokens:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['context'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 77, 768)
      name: StatefulPartitionedCall:0
  outputs['unconditional_context'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 77, 768)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


## Test the encoder SavedModel

In [30]:
tokenizer = SimpleTokenizer()
prompt = "photograph of an astronaut riding a horse"
inputs = tokenizer.encode(prompt)

phrase = inputs + [49407] * (MAX_PROMPT_LENGTH - len(inputs))
# phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)

In [31]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=text_encoder \
  --model_base_path=/content/text_encoder >server.log 2>&1

In [33]:
!cat server.log

[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 230] NET_LOG: Couldn't bind to port 8501
[evhttp_server.cc : 63] NET_LOG: Server has not been terminated. Force termination now.
[evhttp_server.cc : 265] NET_LOG: Server is not running ...
2022-12-21 01:41:29.223397: E tensorflow_serving/model_servers/server.cc:440] Failed to start HTTP Server at localhost:8501


In [34]:
!sudo lsof -i -P -n | grep LISTEN

node          7 root   21u  IPv6  19148      0t0  TCP *:8080 (LISTEN)
kernel_ma    35 root    7u  IPv4  18795      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil    60 root    5u  IPv4  17804      0t0  TCP *:3453 (LISTEN)
colab-fil    60 root    6u  IPv6  17805      0t0  TCP *:3453 (LISTEN)
jupyter-n    82 root    6u  IPv4  17873      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3    1342 root   23u  IPv4  41610      0t0  TCP 127.0.0.1:43909 (LISTEN)
python3    1385 root    3u  IPv4  41978      0t0  TCP 127.0.0.1:22711 (LISTEN)
python3    1385 root    4u  IPv4  41979      0t0  TCP 127.0.0.1:33185 (LISTEN)
python3    1385 root    9u  IPv4  42210      0t0  TCP 127.0.0.1:41729 (LISTEN)
java       2720 root   19u  IPv4  51865      0t0  TCP 127.0.0.1:41333 (LISTEN)
tensorflo  8136 root    5u  IPv4 199864      0t0  TCP *:8500 (LISTEN)
tensorflo  8136 root   12u  IPv4 199872      0t0  TCP *:8501 (LISTEN)
tensorflo 23870 root    5u  IPv4 411892      0t0  TCP *:8500 (LISTEN)


In [35]:
import json

data = json.dumps({"signature_name": "serving_default", "instances": [{"tokens": phrase, "batch_size": 1}]})
print("Data: {} ... {}".format(data[:50], data[len(data) - 52 :]))

Data: {"signature_name": "serving_default", "instances": ... 407, 49407, 49407, 49407, 49407], "batch_size": 1}]}


In [36]:
import requests

headers = {"content-type": "application/json"}
json_response = requests.post(
    "http://localhost:8501/v1/models/text_encoder:predict", data=data, headers=headers
)

In [37]:
json_response = json.loads(json_response.text)

In [39]:
import numpy as np

context = np.array(json_response['predictions'][0]['context'])
print(context.shape)
print(context)

(77, 768)
[[-0.38837659  0.02294353 -0.05219658 ... -0.48988289 -0.306602
   0.06745389]
 [ 0.11565271  0.41449198  2.30493188 ... -1.77114189 -1.01414979
  -0.92086858]
 [-0.43356392 -0.09044611  0.72289908 ...  0.87553692 -1.09482622
  -0.72463357]
 ...
 [-3.0804112  -0.19198492 -0.33464611 ...  0.53937751 -0.15574202
   0.60613984]
 [-3.08857059 -0.24322379 -0.3512004  ...  0.56854761 -0.15004414
   0.59803879]
 [-3.01695275 -0.24591462 -0.30276683 ...  0.5613395  -0.10421869
   0.58553594]]


In [40]:
unconditional_context = np.array(json_response['predictions'][0]['unconditional_context'])
print(unconditional_context.shape)
print(unconditional_context)

(77, 768)
[[-0.38837659  0.02294353 -0.05219658 ... -0.48988289 -0.306602
   0.06745389]
 [-0.3711203  -1.44965601 -0.34011435 ...  0.94886273  0.18672347
  -1.10343707]
 [-0.5107379  -1.46287918 -0.29255477 ...  1.04190361  0.07005903
  -1.02841437]
 ...
 [ 0.50059444 -0.95523065 -0.66102648 ...  1.6012907  -1.06221962
  -0.21908539]
 [ 0.49881172 -0.94508839 -0.66560739 ...  1.64667022 -1.0858376
  -0.20878072]
 [ 0.4923487  -0.81244445 -0.49119091 ...  1.61076951 -1.01735508
  -0.2483795 ]]


# Diffusion Model in SavedModel

In [20]:
test = tf.constant(_ALPHAS_CUMPROD)

In [21]:
test[0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.99915>

In [8]:
from tensorflow import keras
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel

MAX_PROMPT_LENGTH = 77
IMG_HEIGHT = 512
IMG_WIDTH = 512

diffusion_model = DiffusionModel(IMG_HEIGHT, IMG_WIDTH, MAX_PROMPT_LENGTH)
diffusion_model_weights_fpath = keras.utils.get_file(
    origin="https://huggingface.co/fchollet/stable-diffusion/resolve/main/kcv_diffusion_model.h5",
    file_hash="8799ff9763de13d7f30a683d653018e114ed24a6a819667da4f5ee10f9e805fe",
)
diffusion_model.load_weights(diffusion_model_weights_fpath)

3439090152/3439090152 [==============================] - 78s 0us/step


In [9]:
diffusion_model.inputs

[<KerasTensor: shape=(None, 64, 64, 4) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 320) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 77, 768) dtype=float32 (created by layer 'input_1')>]

In [22]:
from keras_cv.models.stable_diffusion.constants import _ALPHAS_CUMPROD

In [23]:
timesteps = tf.range(1, 1000, 1000 // 25)
_ALPHAS_CUMPROD_tt = tf.constant(_ALPHAS_CUMPROD)

In [25]:
timesteps

<tf.Tensor: shape=(25,), dtype=int32, numpy=
array([  1,  41,  81, 121, 161, 201, 241, 281, 321, 361, 401, 441, 481,
       521, 561, 601, 641, 681, 721, 761, 801, 841, 881, 921, 961],
      dtype=int32)>

In [29]:
test = tf.map_fn(lambda t: _ALPHAS_CUMPROD_tt[t], timesteps, dtype=tf.float32)

In [33]:
test

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([0.998296  , 0.96087277, 0.9171379 , 0.86737   , 0.81210774,
       0.7521434 , 0.6884991 , 0.6223854 , 0.55514455, 0.4881804 ,
       0.4228815 , 0.36054322, 0.30229566, 0.24904492, 0.20143245,
       0.15981644, 0.12427604, 0.09463691, 0.0705137 , 0.05136392,
       0.03654652, 0.02537862, 0.01718517, 0.01133791, 0.00728173],
      dtype=float32)>

In [32]:
test[:-1]

<tf.Tensor: shape=(24,), dtype=float32, numpy=
array([0.998296  , 0.96087277, 0.9171379 , 0.86737   , 0.81210774,
       0.7521434 , 0.6884991 , 0.6223854 , 0.55514455, 0.4881804 ,
       0.4228815 , 0.36054322, 0.30229566, 0.24904492, 0.20143245,
       0.15981644, 0.12427604, 0.09463691, 0.0705137 , 0.05136392,
       0.03654652, 0.02537862, 0.01718517, 0.01133791], dtype=float32)>

In [36]:
tf.concat([[1.0], test[:-1]], 0)

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([1.        , 0.998296  , 0.96087277, 0.9171379 , 0.86737   ,
       0.81210774, 0.7521434 , 0.6884991 , 0.6223854 , 0.55514455,
       0.4881804 , 0.4228815 , 0.36054322, 0.30229566, 0.24904492,
       0.20143245, 0.15981644, 0.12427604, 0.09463691, 0.0705137 ,
       0.05136392, 0.03654652, 0.02537862, 0.01718517, 0.01133791],
      dtype=float32)>

In [31]:
test_prev = [1.0] + test[:-1]
test_prev

<tf.Tensor: shape=(24,), dtype=float32, numpy=
array([1.998296 , 1.9608728, 1.9171379, 1.86737  , 1.8121078, 1.7521434,
       1.6884991, 1.6223854, 1.5551445, 1.4881804, 1.4228815, 1.3605433,
       1.3022957, 1.2490449, 1.2014325, 1.1598165, 1.124276 , 1.0946369,
       1.0705137, 1.051364 , 1.0365465, 1.0253786, 1.0171852, 1.0113379],
      dtype=float32)>

In [39]:
i = tf.constant(1)

In [40]:
test[i]

<tf.Tensor: shape=(), dtype=float32, numpy=0.96087277>

In [57]:
import math
import tensorflow as tf

signature_dict = { "context": tf.TensorSpec(shape=[None, 77, 768], dtype=tf.float32, name="context"),
                   "unconditional_context": tf.TensorSpec(shape=[None, 77, 768], dtype=tf.float32, name="unconditional_context"),
                   "num_steps": tf.TensorSpec(shape=[], dtype=tf.int32, name="num_steps"),
                   "batch_size": tf.TensorSpec(shape=[], dtype=tf.int32, name="batch_size")}

def model_exporter(model: tf.keras.Model):
    IMG_HEIGHT = 512
    IMG_WIDTH = 512
    MAX_PROMPT_LENGTH = 77

    SEED = None

    @tf.function
    def _get_timestep_embedding(timestep, batch_size, dim=320, max_period=10000):
        half = dim // 2
        freqs = tf.math.exp(
            -math.log(max_period) * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return tf.repeat(embedding, batch_size, axis=0)

    @tf.function(input_signature=[signature_dict])
    def serving_fn(string_input):
        img_height = round(IMG_HEIGHT / 128) * 128
        img_width = round(IMG_WIDTH / 128) * 128
        unconditional_guidance_scale = 7.5

        batch_size = string_input["batch_size"]
        num_steps = string_input["num_steps"]

        context = string_input["context"]
        unconditional_context = string_input["unconditional_context"]

        latent = tf.random.normal(
            (batch_size, img_height // 8, img_width // 8, 4), seed=SEED
        )

        timesteps = tf.range(1, 1000, 1000 // num_steps)
        _ALPHAS_CUMPROD_t = tf.constant(_ALPHAS_CUMPROD)
        alphas = tf.map_fn(lambda t: _ALPHAS_CUMPROD_t[t], timesteps, dtype=tf.float32)
        alphas_prev = tf.concat([[1.0], alphas[:-1]], 0)

        index = 0
        for t in timesteps:
          latent_prev = latent
          t_emb = _get_timestep_embedding(t, batch_size)
          unconditional_latent = model(
              [latent, t_emb, unconditional_context]
          )
          latent = model([latent, t_emb, context])
          latent = unconditional_latent + unconditional_guidance_scale * (
              latent - unconditional_latent
          )
          a_t, a_prev = alphas[index], alphas_prev[index]
          pred_x0 = (latent_prev - tf.math.sqrt(1 - a_t) * latent) / tf.math.sqrt(a_t)
          latent = latent * tf.math.sqrt(1.0 - a_prev) + tf.math.sqrt(a_prev) * pred_x0
          index = index + 1

        return {"latent": latent}

    return serving_fn

In [58]:
tf.saved_model.save(
    diffusion_model,
    "./diffusion_model/1/",
    signatures={"serving_default": model_exporter(diffusion_model)},
)

In [59]:
!saved_model_cli show --dir diffusion_model/1/ --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['batch_size'] tensor_info:
      dtype: DT_INT32
      shape: ()
      name: serving_default_batch_size:0
  inputs['context'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 77, 768)
      name: serving_default_context:0
  inputs['num_steps'] tensor_info:
      dtype: DT_INT32
      shape: ()
      name: serving_default_num_steps:0
  inputs['unconditional_context'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 77, 768)
      name: serving_default_unconditional_context:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['latent'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 64, 64, 4)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict
